In [ ]:
cd drive/MyDrive/JAIST/Research/RL/movie-lens-simulator

In [ ]:
!make style
!make setup

In [ ]:
from src.data.movielens1M import LoadMovielens1M, PreprocessMovielens1M
from src.dataloader.movielens import MovieLensDataset
from src.dataloader.dataloader import get_MovieLensDataloaders
from src.model.embedding import Simulator, InitialRecommender
from src.trainer.rl_trainer import rl_trainer
from src.trainer.embedding_trainer import Embedding_Trainer

from configs.config import CFG_DICT
from src.utils.file_handler import File_Handler

from src.agent.actor_critic import ActorCriticAgent

import numpy as np
import torch
import pandas as pd

file_handler = File_Handler(log_dir='train_usr/random_split')
logger = file_handler.make_logger_file('src')

In [ ]:
load_rawML1M = LoadMovielens1M(path=CFG_DICT['DATASET']['PATH'])
raw_datas = load_rawML1M.load_data()

preprocessML1M = PreprocessMovielens1M(raw_datas)
preprocess_datas = preprocessML1M.apply_prepocess()

ratings, users, movies, histories = preprocess_datas

In [ ]:

preprocess_datas = ratings, users, movies, histories

device = 'cuda'

user_simulator = Simulator(emb_size=CFG_DICT['USER_SIMULATOR']['DIM_EMB']).to(device)

user_simlator_datas = get_MovieLensDataloaders(
        model_type = 'USER_SIMULATOR',
        processed_data = preprocess_datas,
        data_split = CFG_DICT['USER_SIMULATOR']['DATA_SPLIT']
    )

user_simulator_trainer = Embedding_Trainer(
    model_type = 'USER_SIMULATOR',
    model = user_simulator,
    loaders = user_simlator_datas,
    device = device
    )

user_simulator = user_simulator_trainer.train()
file_handler.save(user_simulator.state_dict(), 'user_simulator.pkl')